In [25]:
import requests
import json
import os

# mettre son appID et sa key puis supprimer.
appid = "a5b42571"
key = os.environ.get("api_adzuna")
url = "http://api.adzuna.com/v1/api/jobs/fr/categories?app_id={0}&app_key={1}&&content-type=application/json".format(appid, key)
req1 = requests.get(url).json()

categories = req1["results"]


tags = [] # j'initie cette liste qui récupèrera les tags qui correspondent aux catégories

for cat in categories:
    tags.append(cat["tag"]) # Pour chaque catégorie je récupère son tag

print(tags)

['accounting-finance-jobs', 'it-jobs', 'sales-jobs', 'customer-services-jobs', 'engineering-jobs', 'hr-jobs', 'healthcare-nursing-jobs', 'hospitality-catering-jobs', 'pr-advertising-marketing-jobs', 'logistics-warehouse-jobs', 'teaching-jobs', 'trade-construction-jobs', 'admin-jobs', 'legal-jobs', 'creative-design-jobs', 'graduate-jobs', 'retail-jobs', 'consultancy-jobs', 'manufacturing-jobs', 'scientific-qa-jobs', 'social-work-jobs', 'travel-jobs', 'energy-oil-gas-jobs', 'property-jobs', 'charity-voluntary-jobs', 'domestic-help-cleaning-jobs', 'maintenance-jobs', 'part-time-jobs', 'other-general-jobs', 'unknown']


Connexion aux bases de données

In [4]:
from sqlalchemy import create_engine, MetaData, text
from pymongo import MongoClient

### MYSQL
username = "ff"
password = os.environ.get("mysql_ff_jobmarket")
hostname = "52.31.100.105"
port = "3306"
database_name = "jobmarket"

db_url = f"mysql+mysqlconnector://{username}:{password}@{hostname}:{port}/{database_name}"
metadata = MetaData()
engine = create_engine(db_url)

### MONGODB
client = MongoClient(
    host="52.31.100.105",
    port = 27017,
    username = "ff",
    password = os.environ.get("mysql_ff_jobmarket")
)

print(client.list_database_names())

['admin', 'config', 'local']


Création de la base de données sur SQL

In [22]:
with engine.connect() as connection:
    connection.execute(text("CREATE TABLE POSTES (id bigint PRIMARY KEY,nom_poste varchar(255),date_creation datetime,ville varchar(255),pays varchar(255),url varchar(255),nom_entreprise varchar(255),salaire int,type_contrat varchar(255),categorie varchar(255),latitude float,longitude float);"))
    

Création de la base de données sur MONGODB

In [ ]:
job = client["job"] #création de la base de données
descriptions = job.create_collection(name="descriptions") # Création de la collection
print(client["job"].list_collection_names())

Récupération des données

In [26]:
countries = ["fr"]
results, success, failed = [], [], [] # Création de 3 listes, une pour les résultats, une pour la journalisation des succés et des échecs
for country in countries:
    for page in range(10, 13):
        url = "https://api.adzuna.com/v1/api/jobs/{3}/search/{0}?app_id={1}&app_key={2}".format(page, appid, key, country)
        req = requests.get(url)
        if req.status_code == 200: # Si réponse positive, alors on récupère les données
            try:
                data = req.json()
                results.append(data["results"])
            except:
                failed.append("{} : {}".format(country, page))
                print(req.status_code, " pour la page ", page, "du pays ", country)
            else:
                success.append("{} : {}".format(country, page))

        else :
            print(req.status_code, " pour la page ", page, "du pays ", country)

print(f"Succès : {len(success)}\nEchec : {len(failed)}")

Succès : 3
Echec : 0


Importation des données V2 Flat

In [30]:
from flatten_json import flatten
success_insert, duplicate_insert, failed_insert = 0, 0, 0
columns = ["id", "title", "created", "location_display_name", "location_area_0", "redirect_url", "company_display_name", "salary_is_predicted", "contract_type", "category_label", "latitude", "longitude"]

job = client["job"] #Connexion à MongoDB
with engine.connect() as connection:
    for page in results:
        for annonce in page:
            annonce_flatten = flatten(annonce)
            
        
            date_creation = str(annonce['created']).replace("T", " ").replace("Z", "")


            text_columns = "("
            text_values = "("
            for column in columns:
                if column in annonce_flatten.keys():
                    text_columns = f"{text_columns}{column}, "
                    if column == "created": #Si c'est une date je formate
                        text_values = f'{text_values}"{annonce_flatten[column].replace("T", " ").replace("Z", "")}", '
                    elif column == "id" or column == "salary_is_predicted" or column == "latitude" or column == "longitude": # Si c'est un nom je récupère seulement
                        text_values = f'{text_values}{annonce_flatten[column]}, '
                    else: # Si c'est du texte, j'ajoute des double quotes
                        text_values = f'{text_values}"{annonce_flatten[column]}", '

            text_columns = text_columns[:-2] + ")"
            text_values = text_values[:-2] + ")"

            replacement = {"title" : "nom_poste", "created" : "date_creation", "location_display_name" : "ville", "location_area_0" : "pays", "redirect_url" : "url", "company_display_name" : "nom_entreprise", "salary_is_predicted" : "salaire", "contract_type" : "type_contrat", "category_label" : "categorie"}
            for key, value in replacement.items():
                text_columns = text_columns.replace(key, value)

            request_text = f"INSERT INTO POSTES {text_columns} VALUES {text_values}"

            mongodb_value = '{ "id":' + annonce['id'] + ', "description":"' + annonce['description'] + '"}'
            mongodb_json = json.loads(mongodb_value)
            


            

        # On ajoute la ligne à la base de données
            try:
                connection.execute(text(request_text))
            except:
                duplicate_insert += 1
            else:
                success_insert += 1
                job.descriptions.insert_one(mongodb_json) # Si ce n'est pas un doublon, alors on importe l'id et la description dans MONGODB

        
    connection.commit()       
    print(f"Success : {success_insert}\nDuplucate : {duplicate_insert}\nFailed : {failed_insert}")
  


Success : 0
Duplucate : 30
Failed : 0


In [29]:
job.descriptions.find_one()

{'_id': ObjectId('662eb6bd9b550b658681c0b0'),
 'id': 4665591089,
 'description': "Randstad vous ouvre toutes les portes de l'emploi : intérim, CDD, CDI. Chaque année, 330 000 collaborateurs (f/h) travaillent dans nos 60 000 entreprises clientes. Rejoignez-nous  Nous recherchons pour le compte de notre client, acteur français de référence dans la gestion et le traitement du document et des nouveaux usages digitaux, un Chargé d'affaires sédentaires (H/F). Rattaché au responsable du service relations clients, le chargé d'affaires sédentaire aura pour mission commerciale de vend…"}